In [2]:
import os

In [3]:
%pwd

'c:\\Users\\91910\\Desktop\\GITProject\\CCDetectionDLproject\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\91910\\Desktop\\GITProject\\CCDetectionDLproject'

In [6]:
#Entity configuration
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
#
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


ModuleNotFoundError: No module named 'cnnClassifier'

In [10]:
pip list

Package                            Version         Editable project location
---------------------------------- --------------- ------------------------------------------
absl-py                            1.4.0
aiohttp                            3.9.3
aiohttp-retry                      2.8.3
aiosignal                          1.3.1
alabaster                          0.7.12
alembic                            1.13.1
amqp                               5.2.0
anaconda-client                    1.7.2
anaconda-navigator                 2.2.0
anaconda-project                   0.9.1
annotated-types                    0.6.0
antlr4-python3-runtime             4.9.3
anyio                              4.2.0
appdirs                            1.4.4
argh                               0.29.4
argon2-cffi                        21.3.0
argon2-cffi-bindings               21.2.0
asn1crypto                         1.5.1
astroid                            2.14.2
astropy                            4.2.1
ast

In [9]:
pip install cnnClassifier

Note: you may need to restart the kernel to use updated packages.


In [11]:
#configuration Manager
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.entity.config_entity import DataIngestionConfig

In [12]:
#configuration Manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [16]:
#Data ingestion Component
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
#will be added in data_ingestion.py 
from cnnClassifier.entity.config_entity import DataIngestionConfig

In [14]:
#Data ingestion Component
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
# Pipeline for(DATA INGESTION) right execution of Downloading the data and extracting it
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-09 02:22:29,960:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-04-09 02:22:29,977:INFO:common:yaml file: params.yaml loaded successfully]
[2024-04-09 02:22:29,980:INFO:common:created directory at: artifacts]
[2024-04-09 02:22:29,983:INFO:common:created directory at: artifacts/data_ingestion]
[2024-04-09 02:22:29,989:INFO:986331348:Downloading data from https://drive.google.com/file/d/1_NwVT-xSB9j-kRqtot_zQFD134792qyf/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1_NwVT-xSB9j-kRqtot_zQFD134792qyf
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1_NwVT-xSB9j-kRqtot_zQFD134792qyf&confirm=t&uuid=fcc71d1a-7a99-428d-8200-ed3f967c83cb
To: c:\Users\91910\Desktop\GITProject\CCDetectionDLproject\artifacts\data_ingestion\data.zip
100%|██████████| 422M/422M [00:28<00:00, 14.6MB/s] 


[2024-04-09 02:23:00,546:INFO:986331348:Downloaded data from https://drive.google.com/file/d/1_NwVT-xSB9j-kRqtot_zQFD134792qyf/view?usp=sharing into file artifacts/data_ingestion/data.zip]
